<a href="https://colab.research.google.com/github/Smfatima/Deep-Learning/blob/main/Computer%20vision/Dog_vs_cat_using_VGG_16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x

In [2]:
#import Libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Activation, BatchNormalization, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings

warnings.simplefilter(action='ignore', category = FutureWarning)
%matplotlib inline

In [ ]:
#Data preparation
os.chdir('/content/drive/MyDrive/Data/dogs-vs-cats.zip (Unzipped Files)/test1.zip (Unzipped Files)')
if os.path.isdir('train/dog') is False:

  os.makedirs('train/dogs')
  os.makedirs('train/cats')
  os.makedirs('valid/dogs')
  os.makedirs('valid/cats')
  os.makedirs('test/dogs')
  os.makedirs('test/cats')

for c in random.choice(len(1000)):
  shutil.move(c, 'train/cat')
for c in random.choice(glob.glob('dog'), 1000):
  shutil.move(c, 'train/dog')
for c in random.sample(glob.glob('cat'), 1000):
  shutil.move(c, 'valid/cat')
for c in random.sample(glob.glob('dog'), 1000):
  shutil.move(c, 'valid/cat')
for c in random.sample(glob.glob('cat'), 1000):
  shutil.move(c, 'test/cat')
for c in random.sample(glob.glob('dog'), 1000):
  shutil.move(c, 'test/dog')

os.chdir('../../')

In [ ]:
train_path = 'data/dogs_vs_cats/train'
valid_path = 'data/dogs_vs_cats/valid'
test_path = 'data/dogs_vs_cats/test'

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\.flow_from_directory(directory = train_path, target_size = (224, 224), classes = ['cat', 'dog'], batch_size = 10)

valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\.flow_from_directory(directory = valid_path, target_size = (224, 224), classes = ['cat', 'dog'], batch_size = 10)

test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.vgg16.preprocess_input)\.flow_from_directory(directory = test_path, target_size = (224, 224), classes = ['cat', 'dog'], batch_size = 10, shuffle = False)


In [ ]:
assert train_batches.n == 1000
assert valid_batches.n == 200
assert test_batches.n == 100

assert train_batches.num_classes == valid_batches.num_classes == test_batches.num_classes == 2

In [ ]:
imgs, labels = next(train_batches)

In [ ]:
def plotImages(images_arr):
  fig, axes = plt.subplot(1, 10, figsize = (20, 20))
  axes = axes.flatten()
  for img, ax in zip(images_arr, axes):
    ax.imshow(img)
    ax.axis('off')
  plt.tight_layout()
  plt.show()
  plotImages(imgs)
  print(labels)

In [ ]:
#build and train a cnn
model = Sequential([
                    Conv2D(filters = 32, kernel_size = (3,3), activation = 'relu', padding = 'same', input_shape = (224, 224, 3)),
                    MaxPool2D(pool_size = (2,2), strides=2),
                    Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', padding = 'same'),
                    MaxPool2D(pool_size = (2,2), strides=2),
                    Flatten(),
                    Dense(units = 2, activation = 'softmax')

])

model.summary()

In [ ]:
model.complie(optimizer = Adam(learning_rate=0.001), loss = 'categorical_crossentropy', metrics = ['accuracy'])

model.fit(x=train_batches, validation_data=valid_batches, epochs=10, verbose=2)

In [ ]:
#Predict
test_imgs, test_labels = next(test_batches)
plotImages(test_imgs)
print(test_labels)

In [ ]:
test_batches.classes

In [ ]:
predictions = model.predict(x=test_batches, verbose=0)
np.round(predictions)

cm = confusion_matrix(y_true=test_batches.classes, y_pred=np.argmax(prediction, axis=-1))

def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion_mstrix', cmap=plt.cm.Blues):
  plt.imshow(cm,interpolation='nearest', cmap=cmap)
  plt.title(title)
  plt.colorbar()

cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm=cm,classes=cm_plot_labels)